# Análise da Estrutuda de Dados dos Produtos Open Finance

### Fontes:



In [19]:
import pandas as pd
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import os
import json

## API de Cartões de Crédito

1. **Consolidação do Metadados**
2. **API dos Produtos de Crédito**
3. **Análise Exploratória dos Dados**
4. **Identificação & Merge de DataFrames**
5. **Conclusões**

## 1. Consolidação do Metadados

In [3]:
API = ['cartao_credito','dados_de_conta','informaçoes_gerais','investimentos','operacoes_de_credito']
MAIN_DIR = f"estrutura_APIs/"
columns_to_drop = ['self','first','prev','next','last','totalRecords','totalPages','requestDateTime']


In [4]:
def get_csv_filenames(apis):
    csv_files = []
    for filename in os.listdir(f'{MAIN_DIR}/{apis}/'):
        if filename.endswith('.csv'):
            csv_files.append(filename[:-4]) # removendo .csv

    return csv_files

file_names = list(map(get_csv_filenames,API))

In [5]:
for file in file_names:
    print(file)

['transaçoes_current', 'creditcard_lists', 'transaçoes_fatura', 'transaçoes', 'creditcard_fatura', 'creditcard_limite', 'creditcard_account_id']
['account_list', 'account_balance', 'account_transactions_current', 'account_limit', 'account_transactions', 'account_id']
['pj_id', 'pj_evaluation', 'pj_financial_relation']
['fixed_balance', 'fund_transactions', 'fixed_credit_id', 'fund_investiments', 'variable_income_list', 'variable_income_id', 'variable_income_transactions', 'fixed_transactions', 'fixed_credit_transactions', 'broker_notes', 'fund_balance', 'treasure_title_list', 'credit_fixed_balance', 'variable_income_balance', 'treasure_title_transactions', 'fixed_investiments', 'treasure_titles_balance', 'credit_fixed_investments', 'fund_id', 'treasure_title_id', 'fixed_id']
['pagamentos_direito_creditorio', 'contratos_emprestimos', 'direitos_creditorios', 'pagamentos_emprestimos', 'adiantamento_parcelas', 'contratos_adiantamento', 'parcelas_emprestimos', 'emprestimos_garantias', 'fina

In [8]:
def data_prep(api,filename):
    # dfs = lambda file: pd.read_csv(f'{MAIN_DIR}{api}/{file}.csv',encoding='latin1')
    df_dict = {}
    for files in filename:
        df = pd.read_csv(f'{MAIN_DIR}{api}/{files}.csv',encoding='latin1')
        if len(df)<=2:
            try:
                df = pd.read_csv(f'{MAIN_DIR}{api}/{files}.csv',encoding='utf-8')
            except Exception as e:
                df = pd.read_csv(f'{MAIN_DIR}{api}/{files}.csv',encoding='latin1')
            df = df.drop(columns=[col for col in df.columns if "Unnamed" in col])
            df_dict[files] = df
        else:
            headers = list(df.iloc[0].dropna())
            headers2 = list(df.iloc[1].dropna())
            headers = list(headers+headers2)
            print(files)
            try:
                df = pd.read_csv(f'{MAIN_DIR}{api}/{files}.csv',header=0,skiprows=1,encoding='utf-8')
                #df = pd.read_csv(f'{MAIN_DIR}{api}/{files}.csv', header=0, skiprows=2, usecols=range(len(headers)), names=headers, encoding='latin1')
            except Exception as e:
                df = pd.read_csv(f'{MAIN_DIR}{api}/{files}.csv',header=0,skiprows=1,encoding='latin1')
                df
                #df = pd.read_csv(f'{MAIN_DIR}{api}/{files}.csv', header=0, skiprows=2, usecols=range(len(headers)), names=headers, encoding='utf-8')
            df.drop(columns=[col for col in df.columns if "Unnamed" in col],inplace=True)
            df_dict[files] = df

    return df_dict

In [172]:
custom_headers = pd.read_csv(f'{MAIN_DIR}{API[0]}/{file_names[0][4]}.csv', skiprows=2, encoding='latin1')
custom_headers.drop(columns=[col for col in custom_headers.columns if "Unnamed" in col],inplace=True)

# Combine the header names from both rows into a single list
# headers = list(custom_headers.iloc[0].dropna())
# headers2 = list(custom_headers.iloc[1].dropna())
# headers = headers+headers2

# Read the CSV file again, this time using the combined column names as headers
#df = pd.read_csv(f'{MAIN_DIR}{API[0]}/{file_names[0][-1]}.csv', header=0, names=headers,skiprows=2, encoding='latin1')
#df = pd.read_csv(f'{MAIN_DIR}{API[0]}/{file_names[0][4]}.csv', header=0, skiprows=2, usecols=range(len(headers)), names=headers, encoding='latin1')
custom_headers

,amount,currency,amount.1,currency.1,type,additionalInfo,amount.2,currency.2,valueType,paymentDate,paymentMode,amount.3,currency.3
0,1000.04,BRL,1000.04,BRL,JUROS_REMUNERATORIOS_ATRASO_PAGAMENTO_FATURA,InformaÃ§Ãµes Adicionais,100000.04,BRL,VALOR_PAGAMENTO_FATURA_PARCELADO,2021-05-21,DEBITO_CONTA_CORRENTE,1000.04,BRL


In [ ]:
def read_json(json_path):
    with open (json_path,'r') as bill_json:
        billid = json.load(bill_json)
    df = pd.json_normalize(billid,record_path=['data'])

    df2 = pd.json_normalize(billid,record_path=['data',['financeCharges'][0]])
    df2.rename(columns={'type':'financeCharges.type',
                   'additionalInfo':'financeCharges.Info',
                   'amount':'financeCharges.amount',
                   'currency':'financeCharges.currency'},inplace=True)

    df3 = pd.json_normalize(billid,record_path=['data',['payments'][0]])
    df3.rename(columns={'valueType':'payments.type',
                   'paymentDate':'payments.paymentDate',
                   'paymentMode':'payments.paymentMode',
                   'amount':'payments.amount',
                   'currency':'payments.currency'},inplace=True)
    return pd.concat([df,df2,df3],axis=1)

In [111]:
def df_dicts(produtos,file_names)-> pd.DataFrame:
    return list(map(data_prep,produtos,file_names))


In [ ]:
def clean(df) -> pd.DataFrame:

    return df

In [160]:
for files in file_names[0]:
    df = pd.read_csv(f'{MAIN_DIR}{API[0]}/{files}.csv',encoding='latin1')
    custom_headers = df.iloc[:2]
    try:
        df = pd.read_csv(f'{MAIN_DIR}{API[0]}/{files}.csv',skiprows=1,encoding='utf-8')
    except Exception as e:
        df = pd.read_csv(f'{MAIN_DIR}{API[0]}/{files}.csv',skiprows=1,encoding='latin1')
    df.drop(columns=[col for col in df.columns if "Unnamed" in col],inplace=True)
    df.drop(columns=[col for col in df.columns if col in columns_to_drop],inplace=True)
    break
df

,transactionId,identificationNumber,transactionName,billId,creditDebitType,transactionType,transactionalAdditionalInfo,paymentType,feeType,feeTypeAdditionalInfo,otherCreditsType,otherCreditsAdditionalInfo,chargeIdentificator,chargeNumber,brazilianAmount,amount,transactionDateTime,billPostDate,payeeMCC
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,amount,amount,NaN,NaN,NaN
1,TXpRMU9UQTROMWhZV2xSU1FUazJSMDl,4453.0,PGTO,MTU0OTU1NjI2NTk4OTRmc2ZhZDRmc2Q1NmZkM,DEBITO,CASHBACK,"qlTN<8h;`DWZ,*=<Stq(eslZ!{$5FX|\JOzb@%Ok",A_VISTA,ANUIDADE,!5fKYxP|C|Sy3zC>!tscEuo1~+UG>F`]csi{cmtx8cb fW...,CREDITO_ROTATIVO,G{C>}GpRm`5pUzwRvMkAp}Lg}HU;f53{Y>X(NAtlzs^D?o...,12.0,12.0,1000.0400,1000.0400,2016-01-29T12:29:03.374Z,2021-05-21,5137.0


### 1.1 Lista de DataFrames

In [176]:
dfs_list = df_dicts(API,file_names)  # Listar todos os datasets relacionados a crédito.

transaçoes_current
transaçoes_fatura
transaçoes
creditcard_fatura
creditcard_limite
creditcard_account_id
account_balance
account_transactions_current
account_limit
account_transactions
pj_id
pj_evaluation
pj_financial_relation
fixed_balance
fund_transactions
fixed_credit_id
variable_income_transactions
fixed_transactions
fixed_credit_transactions
broker_notes
fund_balance
credit_fixed_balance
variable_income_balance
treasure_title_transactions
treasure_titles_balance
treasure_title_id
fixed_id
pagamentos_direito_creditorio
contratos_emprestimos
pagamentos_emprestimos
adiantamento_parcelas
parcelas_emprestimos
direitos_creditorios_id
parcelas_financiamento
pagamentos_financiamentos
parecelas_direitos_creditorios
adiantamento_pagamentos
adiantamentos_id
contratos_financiamentos


In [179]:
dfs_list[0]['creditcard_fatura']

,billId,dueDate,billTotalAmount,billMinimumAmount,isInstalment,financeCharges,payments,self,first,prev,next,last,totalRecords,totalPages,requestDateTime
0,NaN,NaN,amount,amount,NaN,type,valueType,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3459087XXZTR,2021-05-21,1000.0400,1000.0400,False,JUROS_REMUNERATORIOS_ATRASO_PAGAMENTO_FATURA,VALOR_PAGAMENTO_FATURA_PARCELADO,https://api.banco.com.br/open-banking/api/v2/r...,https://api.banco.com.br/open-banking/api/v2/r...,https://api.banco.com.br/open-banking/api/v2/r...,https://api.banco.com.br/open-banking/api/v2/r...,https://api.banco.com.br/open-banking/api/v2/r...,1.0,1.0,2021-05-21T08:30:00Z


### 1.2 Data Cleaning

### 1.2 Cleaned Data

| creditCardAccountId   | brandName     |    companyCnpj | name                 | productType   | productAdditionalInfo                              | creditCardNetwork   | networkAdditionalInfo   |
|:----------------------|:--------------|---------------:|:---------------------|:--------------|:---------------------------------------------------|:--------------------|:------------------------|
| XXZTR3459087          | Organização A | 21128159000166 | Cartão Universitário | OUTROS        | ?45q)>zts3:0rK*,j`^]*5=3L ezjyd[dokF{Jp?0`2LbA+<.= | VISA                | AURA CARD               |


### 2 Identificação Dos Cartões

In [175]:
import requests
from bs4 import BeautifulSoup as bs

In [189]:

url = f'https://openbanking-brasil.github.io/areadesenvolvedor/versions/v1.0.0-rc6.3/#introducao'
response = requests.get(url)

if response.status_code == 200:
    soup = bs(response.content,'html.parser')
    tags = soup.find_all('pre',class_='highlight json tab-json')
    # if tags:
    #     json_schema = json.loads(tags.string)

    #     with open('schema.json', 'w') as json_file:
    #         json.dump(json_schema, json_file, indent=4)

In [190]:
tags[0]

<pre class="highlight json tab-json"><code><span class="p">{</span><span class="w">
  </span><span class="nl">"data"</span><span class="p">:</span><span class="w"> </span><span class="p">{</span><span class="w">
    </span><span class="s2">"..."</span><span class="w">
  </span><span class="p">}</span><span class="w">
</span><span class="p">}</span><span class="w">
</span></code></pre>

In [186]:
tags